In [ ]:
pip install faiss-cpu sentence-transformers pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from pathlib import Path
import re
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from pathlib import Path

# Caminho base da sua pasta
pasta_base = Path("/content/drive/MyDrive/rag-gestores")

# Subpastas específicas
pasta_markdowns = pasta_base / "data/markdowns"
pasta_dados = pasta_base / "data/dados"
pasta_odas = pasta_base / "data/odas"

# Testar se as pastas existem
print("Pastas encontradas:")
print("📁 markdowns:", pasta_markdowns.exists())
print("📁 dados:", pasta_dados.exists())
print("📁 odas:", pasta_odas.exists())

Pastas encontradas:
📁 markdowns: True
📁 dados: True
📁 odas: True


In [ ]:
# Carregar novamente o conteúdo do arquivo corrigido
file_path = Path(pasta_markdowns/"1_Devolutiva_dim_pedag_planejamento.md")
with open(file_path, encoding="utf-8") as f:
    texto = f.read()

In [ ]:
# Função para extrair as seções por rubrica e nível
def extrair_devolutivas(texto):
    # Regex mais tolerante a espaços, quebras de linha e variações
    pattern = r'### Rubrica (\d+) – ([^-]+) – Nível ([^\n]+)\s+\*\*Seus pontos fortes:\*\*\s*(.*?)\s+\*\*O que fazer para avançar\?\*\*\s*(.*?)\s+\*\*Necessidades formativas\*\*\s*(.*?)(?=\n### Rubrica|\Z)'
    matches = re.findall(pattern, texto, re.DOTALL)

    dados = []
    for rubrica_num, titulo, nivel, pontos_fortes, avancar, formativas in matches:
        dados.append({
            'rubrica': f'Rubrica {rubrica_num} - {titulo.strip()}',
            'nivel': nivel.strip(),
            'pontos_fortes': pontos_fortes.strip(),
            'avancar': avancar.strip(),
            'formativas': formativas.strip()
        })
    return pd.DataFrame(dados)

In [ ]:
# Separar os blocos com base no padrão de rubrica
blocos = re.split(r'(?=^### Rubrica \d+ – .+? – Nível .+$)', texto, flags=re.MULTILINE)

dados = []

for bloco in blocos:
    bloco = bloco.strip()
    if not bloco.startswith("### Rubrica"):
        continue

    cabecalho = re.match(r'^### Rubrica (\d+) – (.+?) – Nível (.+)$', bloco, flags=re.MULTILINE)
    if not cabecalho:
        continue

    rubrica_num = cabecalho.group(1).strip()
    rubrica_nome = cabecalho.group(2).strip()
    nivel = cabecalho.group(3).strip()

    conteudo = bloco[cabecalho.end():].strip()

    secao_regex = re.compile(
        r'\*\*Seus pontos fortes\*\*[:]*\s*(.*?)\s*'
        r'\*\*O que fazer para avançar\?\*\*\s*(.*?)\s*'
        r'\*\*Necessidades formativas\*\*\s*(.*)',
        flags=re.DOTALL
    )

    secoes = secao_regex.search(conteudo)
    if not secoes:
        continue

    pontos_fortes = secoes.group(1).strip()
    avancar = secoes.group(2).strip()
    formativas = secoes.group(3).strip()

    dados.append({
        "rubrica": f"Rubrica {rubrica_num} - {rubrica_nome}",
        "nivel": nivel,
        "pontos_fortes": pontos_fortes,
        "avancar": avancar,
        "formativas": formativas
    })

# Gerar DataFrame e exibir
df_devolutivas = pd.DataFrame(dados)

In [ ]:
df_devolutivas

,rubrica,nivel,pontos_fortes,avancar,formativas
0,Rubrica 1 - Sensibilização,Avançar,Muito bem! Você já reconhece a importância do ...,"Para avançar, será necessário estar preparado(...",1. **Diagnóstico escolar para embasar o plane...
1,Rubrica 2 - Exploração,Consolidar,Ótimo! Você já se sente preparado(a) para assu...,"Antes de avançar, será necessário fortalecer s...",1. **Diagnóstico escolar para embasar o plane...
2,Rubrica 2 - Exploração,Avançar,Parabéns! Você já se sente preparado(a) para d...,"Para avançar, será fundamental estar preparado...",1. **Diagnóstico escolar para embasar o plane...
3,Rubrica 3 - Liderança Estratégica,Avançar,Excelente trabalho! Você já se sente preparado...,"Para avançar, será necessário fortalecer o pre...",1. **Interdisciplinariedade e práticas integr...
4,Rubrica 4 - Transformação cultural,Consolidar,Fantástico! Você já se sente preparado(a) para...,Embora este nível reflita uma atuação exemplar...,1. **Interdisciplinariedade e práticas integr...


In [ ]:
# Função para mapear a pontuação para rubrica e nível

def pontuacao_para_rubrica_nivel(pontuacao: int):
    if 0 <= pontuacao <= 5:
        return "Rubrica 1 - Sensibilização", "Consolidar"
    elif 6 <= pontuacao <= 9:
        return "Rubrica 1 - Sensibilização", "Avançar"
    elif 10 <= pontuacao <= 16:
        return "Rubrica 2 - Exploração", "Consolidar"
    elif 17 <= pontuacao <= 21:
        return "Rubrica 2 - Exploração", "Avançar"
    elif 22 <= pontuacao <= 30:
        return "Rubrica 3 - Liderança Estratégica", "Consolidar"
    elif 31 <= pontuacao <= 36:
        return "Rubrica 3 - Liderança Estratégica", "Avançar"
    elif 37 <= pontuacao <= 51:
        return "Rubrica 4 - Transformação cultural", "Consolidar"
    else:
        return None, None  # Fora da faixa prevista

In [ ]:
# Exemplo de uso: pontuação = 17 (esperado: Rubrica 2 - Exploração, Avançar)
rubrica, nivel = pontuacao_para_rubrica_nivel(0)

rubrica, nivel

('Rubrica 1 - Sensibilização', 'Consolidar')

In [ ]:
# Buscar a devolutiva correspondente à rubrica e nível obtidos
devolutiva = df_devolutivas[
    (df_devolutivas['rubrica'] == rubrica) &
    (df_devolutivas['nivel'] == nivel)
]

# Exibir o conteúdo encontrado
if not devolutiva.empty:
    resultado = devolutiva.iloc[0]
    {
        "rubrica": resultado["rubrica"],
        "nivel": resultado["nivel"],
        "pontos_fortes": resultado["pontos_fortes"][:300] + "...",
        "avancar": resultado["avancar"][:300] + "...",
        "formativas": resultado["formativas"][:300] + "..."
    }
else:
    "Devolutiva não encontrada para essa pontuação."

In [ ]:
# Função completa para gerar devolutiva textual a partir da pontuação
def gerar_devolutiva(pontuacao: int, df: pd.DataFrame) -> str:
    rubrica, nivel = pontuacao_para_rubrica_nivel(pontuacao)
    if not rubrica:
        return f"❌ Pontuação {pontuacao} está fora da faixa válida (0 a 51)."

    resultado = df[
        (df["rubrica"] == rubrica) &
        (df["nivel"] == nivel)
    ]

    if resultado.empty:
        return f"❌ Não foi possível encontrar a devolutiva para {rubrica} - {nivel}."

    item = resultado.iloc[0]
    texto = f"""
🔢 **Pontuação:** {pontuacao}
🏷️ **Rubrica:** {item['rubrica']}
📊 **Nível:** {item['nivel']}

---

✅ **Seus pontos fortes:**

{item['pontos_fortes']}

---

📈 **O que fazer para avançar:**

{item['avancar']}

---

📚 **Necessidades formativas:**

{item['formativas']}
"""
    return texto.strip()


In [ ]:
from IPython.display import Markdown
exemplo_pontuacao = 40
texto_md = gerar_devolutiva(exemplo_pontuacao, df_devolutivas)
Markdown(texto_md)

🔢 **Pontuação:** 40
🏷️ **Rubrica:** Rubrica 4 - Transformação cultural
📊 **Nível:** Consolidar

---

✅ **Seus pontos fortes:**

Fantástico! Você já se sente preparado(a) para promover uma cultura escolar em que o planejamento pedagógico seja consistente, colaborativo, contínuo e sustentável. Você se percebe preparado para integrar os objetivos e estratégias pedagógicas com o diagnóstico da escola e as políticas educacionais, fortalecendo a interdisciplinaridade e a inovação. Além disso, você se avalia capacitado(a) para envolver professores, estudantes, famílias e demais atores escolares, criando um ambiente de engajamento e autonomia coletiva.

---

📈 **O que fazer para avançar:**

Embora este nível reflita uma atuação exemplar, é importante assegurar que as práticas sustentáveis e inovadoras continuem evoluindo em resposta às novas demandas educacionais e que a cultura escolar permaneça adaptável e aberta a mudanças futuras. O foco na distribuição da liderança e na criação de capacidade e autonomia da equipe continuará sendo fundamental nesse processo. Da mesma forma, continuará sendo importante a incorporação da recomposição das aprendizagens como parte da cultura de planejamento da escola. Veja a seguir algumas indicações de objetos educacionais e recursos de aprendizagem que podem apoiá-lo nessa tarefa.

---

📚 **Necessidades formativas:**

1.  **Interdisciplinariedade e práticas integradoras**
    *   O que é a interdisplinariedade?
    *   Práticas de promoção da interdisciplinariedade no planejamento coletivo.
2.  **Engajamento da comunidade e comunicação**
    *   Participação dos estudantes no planejamento pedagógico
    *   Papel das famílias e participação no planejamento pedagógico
    *   Comunicação eficaz do planejamento para a comunidade escolar
3.  **Distribuição da liderança e criação de uma cultura sustentável**
    *   Distribuição da liderança, definição de papeis, orientação e acompanhamento da equipe durante o processo de planejamento
4.  **Sensibilização e mobilização para recomposição das aprendizagens**

*   Comunicar a estudantes e famílias o que é recomposição e sua importância, diferenciando-a da recuperação escolar, sensibilizando a comunidade sobre a necessidade de abordar defasagens de anos anteriores sem estigmatizar os estudantes.
*   Como engajar toda a equipe escolar na corresponsabilidade pela recomposição (por exemplo, garantindo que não seja vista apenas como uma tarefa do professor de reforço.)

In [ ]:
# === IMPORTAÇÕES ===
import re
import pandas as pd
from pathlib import Path

# === FUNÇÕES DE SUPORTE ===
def is_devolutiva_file(filename):
    return "Devolutiva" in filename

def get_dimensao_subdimensao_from_filename(filename):
    partes = filename.stem.split("_")
    if len(partes) >= 5:
        return partes[3], "_".join(partes[4:])
    return "desconhecida", "desconhecida"

# === PARSER ROBUSTO PARA DEVOLUTIVAS ===
def extrair_blocos_devolutiva_final(markdown_text):
    blocos = re.split(r'(?=^### Rubrica \d+ [\u2013\-] .+? [\u2013\-] Nível .+$)', markdown_text, flags=re.MULTILINE)
    registros = []

    for bloco in blocos:
        bloco = bloco.strip()
        if not bloco.startswith("### Rubrica"):
            continue

        cabecalho = re.match(r'^### Rubrica (\d+) [\u2013\-] (.+?) [\u2013\-] Nível (.+)$', bloco, flags=re.MULTILINE)
        if not cabecalho:
            continue

        rubrica_num = cabecalho.group(1).strip()
        rubrica_nome = cabecalho.group(2).strip()
        nivel = cabecalho.group(3).strip()

        conteudo = bloco[cabecalho.end():].strip()

        # 🔧 Aqui está o parser mais robusto das seções
        partes = re.split(
            r'\*\*\s*(Seus pontos fortes|O que fazer para avançar\??|Necessidades formativas)\s*\*\*[\s:\uFF1A]*\n*',
            conteudo,
            flags=re.IGNORECASE
        )

        secoes = {}
        for i in range(1, len(partes) - 1, 2):
            chave = partes[i].strip().lower()
            valor = partes[i + 1].strip()
            secoes[chave] = valor

        registros.append({
            "rubrica": f"Rubrica {rubrica_num} - {rubrica_nome}",
            "nivel": nivel,
            "pontos_fortes": secoes.get("seus pontos fortes", ""),
            "avancar": secoes.get("o que fazer para avançar", ""),
            "formativas": secoes.get("necessidades formativas", "")
        })

    return registros

def processar_devolutivas_em_pasta_final(pasta_markdowns: Path):
    arquivos_md = list(pasta_markdowns.rglob("*.md"))
    todos_registros = []

    for arquivo in arquivos_md:
        if not is_devolutiva_file(arquivo.name):
            continue

        with open(arquivo, encoding="utf-8") as f:
            texto = f.read()

        registros = extrair_blocos_devolutiva_final(texto)
        dimensao, subdimensao = get_dimensao_subdimensao_from_filename(arquivo)

        for r in registros:
            r["dimensao"] = dimensao
            r["subdimensao"] = subdimensao
            r["arquivo_origem"] = arquivo.name
            todos_registros.append(r)

    return pd.DataFrame(todos_registros)

# === MAPEAMENTO DE PONTUAÇÃO ===
def pontuacao_para_rubrica_nivel(pontuacao: int):
    if 0 <= pontuacao <= 5:
        return "Rubrica 1 - Sensibilização", "Consolidar"
    elif 6 <= pontuacao <= 9:
        return "Rubrica 1 - Sensibilização", "Avançar"
    elif 10 <= pontuacao <= 19:
        return "Rubrica 2 - Exploração", "Consolidar"
    elif 20 <= pontuacao <= 27:
        return "Rubrica 2 - Exploração", "Avançar"
    elif 28 <= pontuacao <= 36:
        return "Rubrica 3 - Liderança Estratégica", "Consolidar"
    elif 37 <= pontuacao <= 42:
        return "Rubrica 3 - Liderança Estratégica", "Avançar"
    elif 43 <= pontuacao <= 48:
        return "Rubrica 4 - Transformação cultural", "Consolidar"
    else:
        return None, None

# === GERADOR DE DEVOLUTIVA ===
def gerar_devolutiva_dimensao(pontuacao: int, dimensao: str, subdimensao: str, df: pd.DataFrame) -> str:
    rubrica, nivel = pontuacao_para_rubrica_nivel(pontuacao)
    if not rubrica:
        return f"❌ Pontuação {pontuacao} está fora da faixa válida."

    df_filtrado = df[
        (df["dimensao"] == dimensao) &
        (df["subdimensao"] == subdimensao)
    ]

    resultado = df_filtrado[
        (df_filtrado["rubrica"] == rubrica) &
        (df_filtrado["nivel"] == nivel)
    ]

    if resultado.empty:
        return f"❌ Não foi encontrada a devolutiva para {rubrica} - {nivel} na subdimensão '{subdimensao}'."

    item = resultado.iloc[0]
    return f"""
🔢 **Pontuação:** {pontuacao}
📂 **Dimensão:** {dimensao}
📁 **Subdimensão:** {subdimensao}
🏷️ **Rubrica:** {item['rubrica']}
📊 **Nível:** {item['nivel']}

---

✅ **Seus pontos fortes:**

{item['pontos_fortes']}

---

📈 **O que fazer para avançar:**

{item['avancar']}

---

📚 **Necessidades formativas:**

{item['formativas']}
""".strip()

In [ ]:
# === EXEMPLO DE USO ===
pasta_corrigida = pasta_markdowns
df_todas_devolutivas = processar_devolutivas_em_pasta_final(pasta_corrigida)
print(gerar_devolutiva_dimensao(0, "pedag", "planejamento", df_todas_devolutivas))

🔢 **Pontuação:** 0
📂 **Dimensão:** pedag
📁 **Subdimensão:** planejamento
🏷️ **Rubrica:** Rubrica 1 - Sensibilização
📊 **Nível:** Consolidar

---

✅ **Seus pontos fortes:**



---

📈 **O que fazer para avançar:**



---

📚 **Necessidades formativas:**

1.  **Conceitos básicos de planejamento pedagógico**

*   O que o planejamento pedagógico e qual é sua importância?
*   O que contempla um planejamento pedagógico de qualidade? (pode incluir roteiros, exemplos etc.)

1.  **Etapas e calendário de planejamento**

*   Quais são as etapas do ciclo de planejamento?
*   Como elaborar um calendário e instituir rotinas de planejamento coletivo e individual ao longo do ano? (pode incluir ferramentas)

**Fundamentos conceituais e normativos da recomposição de aprendizagens**

Definição da recomposição das aprendizagens e sua diferença em relação à recuperação escolar.

Marco legal e normativo: Pacto Nacional pela Recomposição das Aprendizagens (MEC), diretrizes do CNE e normativas estaduais e munic

In [ ]:
# === IMPORTAÇÕES ===
import re
import pandas as pd
from pathlib import Path

# === FUNÇÕES DE SUPORTE ===
def is_devolutiva_file(filename):
    return "Devolutiva" in filename

def get_dimensao_subdimensao_from_filename(filename):
    partes = filename.stem.split("_")
    if len(partes) >= 5:
        return partes[3], "_".join(partes[4:])
    return "desconhecida", "desconhecida"

# === PARSER ROBUSTO COM REGEX FLEXÍVEL PARA DEVOLUTIVAS ===
def extrair_blocos_devolutiva(markdown_text):
    blocos = re.split(r'(?=^### Rubrica \d+ [\u2013\-] .+? [\u2013\-] Nível .+$)', markdown_text, flags=re.MULTILINE)
    registros = []

    for bloco in blocos:
        bloco = bloco.strip()
        if not bloco.startswith("### Rubrica"):
            continue

        cabecalho = re.match(r'^### Rubrica (\d+) [\u2013\-] (.+?) [\u2013\-] Nível (.+)$', bloco, flags=re.MULTILINE)
        if not cabecalho:
            continue

        rubrica_num = cabecalho.group(1).strip()
        rubrica_nome = cabecalho.group(2).strip()
        nivel = cabecalho.group(3).strip()

        # Correções específicas de nome (se necessário)
        if rubrica_num == "3" and rubrica_nome.lower() == "liderança":
            rubrica_nome = "Liderança Estratégica"
        if rubrica_num == "4" and rubrica_nome.lower() == "sensibilização":
            rubrica_nome = "Transformação cultural"

        conteudo = bloco[cabecalho.end():].strip()

        # Inicializar com strings vazias
        secoes = {
            "seus pontos fortes": "",
            "o que fazer para avançar": "",
            "necessidades formativas": ""
        }

        # Detectar seções
        matches = list(re.finditer(
            r'\*\*\s*(Seus pontos fortes|O que fazer para avançar\??|Necessidades formativas)[\s:：]*\*\*',
            conteudo,
            flags=re.IGNORECASE
        ))

        for idx, match in enumerate(matches):
            titulo_bruto = match.group(1).strip().lower().replace("?", "")
            inicio = match.end()
            fim = matches[idx + 1].start() if idx + 1 < len(matches) else len(conteudo)
            texto_bloco = conteudo[inicio:fim].strip()
            secoes[titulo_bruto] = texto_bloco

        registros.append({
            "rubrica": f"Rubrica {rubrica_num} - {rubrica_nome}",
            "nivel": nivel,
            "pontos_fortes": secoes["seus pontos fortes"],
            "avancar": secoes["o que fazer para avançar"],
            "formativas": secoes["necessidades formativas"]
        })

    return registros

# === PROCESSAMENTO DE TODOS OS ARQUIVOS ===
def processar_devolutivas_em_pasta_final(pasta_markdowns: Path):
    arquivos_md = list(pasta_markdowns.rglob("*.md"))
    todos_registros = []

    for arquivo in arquivos_md:
        if not is_devolutiva_file(arquivo.name):
            continue

        with open(arquivo, encoding="utf-8") as f:
            texto = f.read()

        registros = extrair_blocos_devolutiva(texto)
        dimensao, subdimensao = get_dimensao_subdimensao_from_filename(arquivo)

        for r in registros:
            r["dimensao"] = dimensao
            r["subdimensao"] = subdimensao
            r["arquivo_origem"] = arquivo.name
            todos_registros.append(r)

    return pd.DataFrame(todos_registros)

# === MAPEAMENTO DE PONTUAÇÃO ===
def pontuacao_para_rubrica_nivel(pontuacao: int):
    if 0 <= pontuacao <= 4:
        return "Rubrica 1 - Sensibilização", "Consolidar"
    elif 5 <= pontuacao <= 9:
        return "Rubrica 1 - Sensibilização", "Avançar"
    elif 10 <= pontuacao <= 16:
        return "Rubrica 2 - Exploração", "Consolidar"
    elif 17 <= pontuacao <= 21:
        return "Rubrica 2 - Exploração", "Avançar"
    elif 22 <= pontuacao <= 29:
        return "Rubrica 3 - Liderança Estratégica", "Consolidar"
    elif 30 <= pontuacao <= 36:
        return "Rubrica 3 - Liderança Estratégica", "Avançar"
    elif 37 <= pontuacao <= 45:
        return "Rubrica 4 - Transformação cultural", "Consolidar"
    else:
        return None, None

# === GERADOR DE TEXTO DA DEVOLUTIVA FINAL ===
def gerar_devolutiva_dimensao(pontuacao: int, dimensao: str, subdimensao: str, df: pd.DataFrame) -> str:
    rubrica, nivel = pontuacao_para_rubrica_nivel(pontuacao)
    if not rubrica:
        return f"❌ Pontuação {pontuacao} está fora da faixa válida."

    df_filtrado = df[
        (df["dimensao"] == dimensao) & (df["subdimensao"] == subdimensao)
    ]

    resultado = df_filtrado[
        (df_filtrado["rubrica"] == rubrica) & (df_filtrado["nivel"] == nivel)
    ]

    if resultado.empty:
        return f"❌ Não foi encontrada a devolutiva para {rubrica} - {nivel} na subdimensão '{subdimensao}'."

    item = resultado.iloc[0]
    return f"""
🔢 **Pontuação:** {pontuacao}
📂 **Dimensão:** {dimensao}
📁 **Subdimensão:** {subdimensao}
🏷️ **Rubrica:** {item['rubrica']}
📊 **Nível:** {item['nivel']}

---

✅ **Seus pontos fortes:**

{item['pontos_fortes']}

---

📈 **O que fazer para avançar:**

{item['avancar']}

---

📚 **Necessidades formativas:**

{item['formativas']}
""".strip()

In [ ]:
# === 1. IMPORTAR PARSER E GERAR O DATAFRAME ===
from pathlib import Path

# Atualize o caminho com o local correto da sua pasta
pasta_markdowns = Path("/content/drive/MyDrive/rag-gestores/data/markdowns")
df_todas_devolutivas = processar_devolutivas_em_pasta_final(pasta_markdowns)

# === 2. INTERFACE INTERATIVA COM IPYWIDGETS ===
import ipywidgets as widgets
from IPython.display import display, Markdown

# Dropdown de dimensão
dimensoes_unicas = df_todas_devolutivas['dimensao'].unique().tolist()
dropdown_dimensao = widgets.Dropdown(
    options=dimensoes_unicas,
    description='Dimensão:',
    value=dimensoes_unicas[0],
    style={'description_width': 'initial'}
)

# Dropdown de subdimensão (dinâmico)
dropdown_subdimensao = widgets.Dropdown(
    options=[],
    description='Subdimensão:',
    style={'description_width': 'initial'}
)

def atualizar_subdimensoes(*args):
    subdimensoes = df_todas_devolutivas[
        df_todas_devolutivas['dimensao'] == dropdown_dimensao.value
    ]['subdimensao'].unique().tolist()
    dropdown_subdimensao.options = subdimensoes
    if subdimensoes:
        dropdown_subdimensao.value = subdimensoes[0]

dropdown_dimensao.observe(atualizar_subdimensoes, names='value')
atualizar_subdimensoes()

# Slider de pontuação
entrada_pontuacao = widgets.IntSlider(
    value=17,
    min=0,
    max=51,
    step=1,
    description='Pontuação:',
    style={'description_width': 'initial'},
    continuous_update=False
)

# Botão + área de saída
botao_gerar = widgets.Button(description='Gerar devolutiva', button_style='success')
output = widgets.Output()

# Função de clique do botão
def ao_clicar_botao(b):
    with output:
        output.clear_output()
        texto = gerar_devolutiva_dimensao(
            pontuacao=entrada_pontuacao.value,
            dimensao=dropdown_dimensao.value,
            subdimensao=dropdown_subdimensao.value,
            df=df_todas_devolutivas
        )
        display(Markdown("## 📄 **Devolutiva personalizada:**"))
        display(Markdown(texto))

botao_gerar.on_click(ao_clicar_botao)

# Mostrar tudo
display(dropdown_dimensao, dropdown_subdimensao, entrada_pontuacao, botao_gerar, output)

Dropdown(description='Dimensão:', options=('pedag', 'pessoal'), style=DescriptionStyle(description_width='init…

Dropdown(description='Subdimensão:', options=('monitoriamento', 'desenvolvimento', 'implementacao', 'protecao'…

IntSlider(value=17, continuous_update=False, description='Pontuação:', max=51, style=SliderStyle(description_w…

Button(button_style='success', description='Gerar devolutiva', style=ButtonStyle())

Output()

In [ ]:
def get_rubrica_result(dimensao, subdimensao, rubrica_numero, pontuacao, faixas):
 # Filtra as faixas que correspondem à rubrica desejada.
    registros = [
        f for f in faixas
        if f["dimensao"] == dimensao and
           f["subdimensao"] == subdimensao and
           f["rubrica_numero"] == rubrica_numero
    ]
# Se não encontrar registros, retorna aviso.
    if not registros:
        return "Rubrica não encontrada."

    # Verifica em qual subfaixa a pontuação do usuário se encaixa.
    for reg in registros:
        if reg["subfaixa_min"] <= pontuacao <= reg["subfaixa_max"]:
            # Retorna o texto apropriado.
            return f"Você atingiu o nível '{reg['tipo_faixa']}' na rubrica '{reg['rubrica_nome']}'."

In [ ]:
# === 1. IMPORTAÇÕES ===
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

# === 2. CARREGAR ARQUIVOS ===
df_devolutivas = pd.read_csv("Devolutivas.csv", sep=";")
df_rubricas = pd.read_csv("Rubricas.csv", sep=";")

# Corrigir nome da coluna
df_devolutivas = df_devolutivas.rename(columns={
    "Necessidaes formativas": "Necessidades formativas"
})

# === 3. FUNÇÕES ===

def encontrar_rubrica(pontuacao, dimensao, subdimensao):
    candidatos = df_rubricas[
        (df_rubricas['dimensao'] == dimensao) &
        (df_rubricas['subdimensao'] == subdimensao) &
        (df_rubricas['faixa_total_min'] <= pontuacao) &
        (df_rubricas['faixa_total_max'] >= pontuacao)
    ]
    if candidatos.empty:
        return None, None, None

    rubrica_numero = candidatos.iloc[0]['rubrica_numero']
    rubrica_nome = candidatos.iloc[0]['rubrica_nome']

    faixa_escolhida = candidatos[
        (candidatos['subfaixa_min'] <= pontuacao) &
        (candidatos['subfaixa_max'] >= pontuacao)
    ]
    if faixa_escolhida.empty:
        return rubrica_numero, rubrica_nome, None

    tipo_faixa = faixa_escolhida.iloc[0]['tipo_faixa']
    return rubrica_numero, rubrica_nome, tipo_faixa

def gerar_texto_devolutiva_markdown(pontuacao, dimensao, subdimensao):
    rubrica_numero, rubrica_nome, tipo_faixa = encontrar_rubrica(pontuacao, dimensao, subdimensao)

    if not rubrica_numero or not tipo_faixa:
        return f"❌ Pontuação {pontuacao} está fora da faixa válida para a dimensão '{dimensao}' e subdimensão '{subdimensao}'."

    rubrica_nome_completo = f"{rubrica_nome} – Nível {tipo_faixa}"

    devolutiva = df_devolutivas[
        (df_devolutivas['Dimensão'] == dimensao) &
        (df_devolutivas['Subdimensão'] == subdimensao) &
        (df_devolutivas['Rubrica numero'] == rubrica_numero) &
        (df_devolutivas['Rubrica nome'] == rubrica_nome_completo)
    ]

    if devolutiva.empty:
        return f"❌ Texto da devolutiva não encontrado para Rubrica {rubrica_numero} - {rubrica_nome_completo}."

    item = devolutiva.iloc[0]

    return f"""
## 📄 **Devolutiva personalizada:**

🔢 **Pontuação:** {pontuacao}
📂 **Dimensão:** {dimensao}
📁 **Subdimensão:** {subdimensao}
🏷️ **Rubrica:** Rubrica {rubrica_numero} - {rubrica_nome}
📊 **Nível:** {tipo_faixa}

---

✅ **Seus pontos fortes:**

{item['Pontos fortes']}

---

📈 **O que fazer para avançar:**

{item['O que fazer para avançar']}

---

📚 **Necessidades formativas:**

{item['Necessidades formativas']}
""".strip()

# === 4. INTERFACE ===

dimensoes_unicas = df_devolutivas['Dimensão'].unique().tolist()
dropdown_dimensao = widgets.Dropdown(
    options=dimensoes_unicas,
    description='Dimensão:',
    value=dimensoes_unicas[0],
    style={'description_width': 'initial'}
)

dropdown_subdimensao = widgets.Dropdown(
    options=[],
    description='Subdimensão:',
    style={'description_width': 'initial'}
)

def atualizar_subdimensoes(*args):
    subdimensoes = df_devolutivas[
        df_devolutivas['Dimensão'] == dropdown_dimensao.value
    ]['Subdimensão'].unique().tolist()
    dropdown_subdimensao.options = subdimensoes
    if subdimensoes:
        dropdown_subdimensao.value = subdimensoes[0]

dropdown_dimensao.observe(atualizar_subdimensoes, names='value')
atualizar_subdimensoes()

entrada_pontuacao = widgets.IntSlider(
    value=17,
    min=0,
    max=51,
    step=1,
    description='Pontuação:',
    style={'description_width': 'initial'},
    continuous_update=False
)

botao_gerar = widgets.Button(description='Gerar devolutiva', button_style='success')
output = widgets.Output()

def ao_clicar_botao(b):
    with output:
        output.clear_output()
        print("⏳ Gerando devolutiva...")
        texto = gerar_texto_devolutiva_markdown(
            pontuacao=entrada_pontuacao.value,
            dimensao=dropdown_dimensao.value,
            subdimensao=dropdown_subdimensao.value
        )
        output.clear_output()
        display(Markdown(texto))

botao_gerar.on_click(ao_clicar_botao)

display(dropdown_dimensao, dropdown_subdimensao, entrada_pontuacao, botao_gerar, output)


Dropdown(description='Dimensão:', options=('Dimensão pedagógica', 'Dimensão pessoal-relacional'), style=Descri…

Dropdown(description='Subdimensão:', options=('Planejamento pedagógico', 'Implementação do processo de ensino …

IntSlider(value=17, continuous_update=False, description='Pontuação:', max=51, style=SliderStyle(description_w…

Button(button_style='success', description='Gerar devolutiva', style=ButtonStyle())

Output()

In [ ]:
caminho_csv = "/content/drive/MyDrive/rag-gestores/data/odas/repositorio-odas.csv"
df_odas = pd.read_csv(caminho_csv, sep=";", encoding="utf-8")
df_odas.head()

,Título,Link fixo,Data de publicação,Instituições,Fonte,Data de acesso à fonte,Suporte,Duração/Tamanho,Local de Publicação,Idiomas,...,Temas,Dimensões,Palavras-chave - Termos do vocabulário controlado,Direitos autorais - Tipo,Temas e termos do vocabulário controlado,Localizações,Datas,Seção do Observatório,Detalhes da Catalogação,id
0,A qualidade da educação no horizonte da proteç...,a-qualidade-da-educacao-no-horizonte-da-protec...,01/01/2020 00:00,NaN,https://www.scielo.br/j/cp/a/3gRPNHGZhvgfYGkYf...,NaN,9293d806-8419-4127-b442-6437549105e5,19 páginas,NaN,d86ea641-fa95-4d83-be26-ce1b70bcf476,...,c6424ec9-408e-46fa-8c6e-2b215bcd0d42|068f159c-...,faf99f35-3654-4278-8f62-773faea7f61a,f47109d2-4463-4d37-a00e-e14208a6e499|4a108e05-...,a9d98290-8c70-4978-b9ba-f1f46f688ea1,c6424ec9-408e-46fa-8c6e-2b215bcd0d42|068f159c-...,NaN,NaN,cedoc,,fc7ad301-e959-4f89-a94a-6e66beaf2a88
1,Relatório de pesquisa – Papel da Supervisora E...,relatorio-de-pesquisa-papel-da-supervisora-esc...,01/01/2020 00:00,"Relação:Editora,Instituição:#067b3557-7a4f-4dc...",,NaN,9293d806-8419-4127-b442-6437549105e5,48 páginas,84bad139-d7a7-40ba-9cc8-561764d7f12e,d86ea641-fa95-4d83-be26-ce1b70bcf476,...,068f159c-3265-4192-b288-bcdbfcdce78d|5f582d23-...,2d33481d-710a-449e-8a73-aa0e2b9efbac|faf99f35-...,4bf88502-b339-4eb1-94ad-dfcfb856f1a0|a74cdf68-...,0c266d06-2126-43e9-aadc-2113eb66a297,068f159c-3265-4192-b288-bcdbfcdce78d|5f582d23-...,NaN,NaN,cedoc,,3b29481f-b55a-499e-8f7d-8182c020b26a
2,Relatório de pesquisa – Avaliação responsiva e...,relatorio-de-pesquisa-avaliacao-responsiva-em-...,01/01/2020 00:00,NaN,,NaN,9293d806-8419-4127-b442-6437549105e5,54 páginas,84bad139-d7a7-40ba-9cc8-561764d7f12e,d86ea641-fa95-4d83-be26-ce1b70bcf476,...,964e45f7-2ab1-40d5-ae37-0f3b772841ba|17a7a4b9-...,325127b6-0cb8-42f4-830a-3f233aa5c897|faf99f35-...,601d4133-e752-4d41-95cf-7d37c5c39d41|4bf88502-...,0c266d06-2126-43e9-aadc-2113eb66a297,964e45f7-2ab1-40d5-ae37-0f3b772841ba|17a7a4b9-...,NaN,NaN,cedoc,,fdf53695-4aa8-4071-9223-0c39a38a03b3
3,COVID-19 - Impacto Fiscal na Educação Básica: ...,covid-19-impacto-fiscal-na-educacao-basica-o-c...,01/01/2020 00:00,"Relação:Autora,Instituição:#b9df277d-f409-4c45...",,NaN,9293d806-8419-4127-b442-6437549105e5,18 páginas,84bad139-d7a7-40ba-9cc8-561764d7f12e,d86ea641-fa95-4d83-be26-ce1b70bcf476,...,6c633342-21e7-4783-b253-1ad0142366ad|068f159c-...,faf99f35-3654-4278-8f62-773faea7f61a,3b4ca857-b1af-466e-8d0e-4af9d776fd39|4a7e9cac-...,a9d98290-8c70-4978-b9ba-f1f46f688ea1,6c633342-21e7-4783-b253-1ad0142366ad|068f159c-...,NaN,NaN,cedoc,,6e9abf49-4209-443d-b113-e98a575e77ae
4,Ensinando pensamento crítico: sabedoria prática,tfr-ensinando-pensamento-critico-sabedoria-pra...,01/01/2020 00:00,"Relação:Editora,Instituição:#4051cc9f-fe4a-461...",,NaN,9293d806-8419-4127-b442-6437549105e5,294 páginas,97f35add-f213-497c-8419-077d1f5e06cd,d86ea641-fa95-4d83-be26-ce1b70bcf476,...,17a7a4b9-a285-4199-b013-5518fbdd6d3a|b4da8735-...,faf99f35-3654-4278-8f62-773faea7f61a,7f73b330-81f3-4d2a-90e5-4ace2e079512|6c93daa5-...,8abcc139-f6b8-42ca-b94a-d78df9ffdbd1,17a7a4b9-a285-4199-b013-5518fbdd6d3a|b4da8735-...,NaN,NaN,cedoc,,bdaef6c0-c71a-48f5-9d7f-1abc8e40222a


In [ ]:
caminho_csv = "repositoriorelat-odas.xlsx"
df_odas = pd.read_csv(caminho_csv, sep=";", encoding="utf-8")
df_odas

FileNotFoundError: [Errno 2] No such file or directory: 'repositorio-odas.xlsx'

## RAG

In [ ]:
# === IMPORTAÇÕES ===
import pandas as pd
import numpy as np
import faiss
import pickle
import re
from sentence_transformers import SentenceTransformer

# === 1. Carregar os dados do repositório ===
caminho_csv = "/content/drive/MyDrive/rag-gestores/data/odas/repositorio-odas.csv"
df_odas = pd.read_csv(caminho_csv, sep=";", encoding="utf-8")

# === 2. Interpretar a coluna Duração/Tamanho ===
def interpretar_duracao(duracao):
    if pd.isna(duracao) or duracao.strip().lower() in ['s/d', '']:
        return "⏱️ Duração não informada"
    if "min" in duracao:
        minutos = int(re.findall(r"\d+", duracao)[0])
        if minutos <= 5:
            return f"🎥 {duracao} (vídeo curto)"
        elif minutos <= 20:
            return f"🎬 {duracao} (vídeo médio)"
        else:
            return f"🧑‍🏫 {duracao} (aula ou curso)"
    elif "página" in duracao:
        paginas = int(re.findall(r"\d+", duracao)[0])
        if paginas <= 3:
            return f"📄 {duracao} (texto rápido)"
        elif paginas <= 10:
            return f"📘 {duracao} (leitura média)"
        else:
            return f"📚 {duracao} (artigo extenso)"
    return f"📌 {duracao}"

df_odas["Descricao_duracao"] = df_odas["Duração/Tamanho"].apply(interpretar_duracao)

# === 3. Criar campo de texto completo para embeddings ===
def preparar_texto_oda(row):
    partes = [
        f"Título: {row.get('Título', '')}",
        f"Resumo: {row.get('Resumo', '')}",
        f"Temas: {row.get('Temas', '')}",
        f"Palavras-chave: {row.get('Palavras-chave - Termos do vocabulário controlado', '')}",
        f"Dimensões: {row.get('Dimensões', '')}",
        f"Suporte: {row.get('Suporte', '')}",
        f"Duração: {row.get('Descricao_duracao', '')}",
    ]
    return "\n".join([p for p in partes if pd.notnull(p)])

df_odas["texto_completo"] = df_odas.apply(preparar_texto_oda, axis=1)

# === 4. Gerar embeddings com SentenceTransformer ===
modelo = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = modelo.encode(df_odas["texto_completo"].tolist(), show_progress_bar=True)

# === 5. Criar índice FAISS ===
dimensao = embeddings.shape[1]
index = faiss.IndexFlatL2(dimensao)
index.add(np.array(embeddings).astype("float32"))

# === 6. Guardar metadados úteis para exibição futura ===
metadados = df_odas[[
    "Título",
    "Link fixo",            # nome do material
    "Fonte",                # link verdadeiro
    "Resumo",
    "Suporte",
    "Dimensões",
    "Descricao_duracao",
    "texto_completo"
]].reset_index(drop=True)

# === 7. Salvar o índice e metadados para reutilização ===
with open("metadados_odas.pkl", "wb") as f:
    pickle.dump(metadados, f)
faiss.write_index(index, "odas_index.faiss")

# === 8. Verificação final ===
print("Index size:", index.ntotal)
print(metadados.head(3))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/76 [00:00<?, ?it/s]

Index size: 2422
                                              Título  \
0  A qualidade da educação no horizonte da proteç...   
1  Relatório de pesquisa – Papel da Supervisora E...   
2  Relatório de pesquisa – Avaliação responsiva e...   

                                           Link fixo  \
0  a-qualidade-da-educacao-no-horizonte-da-protec...   
1  relatorio-de-pesquisa-papel-da-supervisora-esc...   
2  relatorio-de-pesquisa-avaliacao-responsiva-em-...   

                                               Fonte  \
0  https://www.scielo.br/j/cp/a/3gRPNHGZhvgfYGkYf...   
1                                                      
2                                                      

                                              Resumo  \
0  <p>O artigo pretende problematizar o conceito ...   
1  Esta pesquisa, tem como finalidade investigar ...   
2  O presente estudo de avaliação responsiva pret...   

                                Suporte  \
0  9293d806-8419-4127-b442-6437549105e5 

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import numpy as np
import pandas as pd

# === CRIAÇÃO DA COLUNA Descricao_duracao ===
def classificar_duracao(valor):
    if pd.isna(valor):
        return "⏱️ Duração não informada"
    texto = str(valor).lower()
    if "hora" in texto or ":" in texto:
        return f"🎥 {valor} (vídeo)"
    elif "página" in texto or texto.isdigit():
        return f"📚 {valor} (artigo)"
    return f"⏱️ {valor}"

if "Descricao_duracao" not in df_odas.columns:
    df_odas["Descricao_duracao"] = df_odas["Duração/Tamanho"].apply(classificar_duracao)

# === FUNÇÃO PARA EMBEDDING A PARTIR DA DEVOLUTIVA ===
def gerar_embedding_para_rag(texto: str) -> np.ndarray:
    # Tenta extrair parte mais relevante para recomendação
    trecho = ""
    if "**Necessidades formativas:**" in texto:
        trecho = texto.split("**Necessidades formativas:**")[-1].strip()
    elif "**Seus pontos fortes:**" in texto:
        trecho = texto.split("**Seus pontos fortes:**")[-1].strip()
    else:
        trecho = texto  # fallback
    return modelo.encode([trecho])

# === DROPDOWNS INTERATIVOS ===
dimensoes_unicas = df_todas_devolutivas['dimensao'].unique().tolist()

dropdown_dimensao = widgets.Dropdown(
    options=dimensoes_unicas,
    description='Dimensão:',
    value=dimensoes_unicas[0],
    style={'description_width': 'initial'}
)

dropdown_subdimensao = widgets.Dropdown(
    options=[],
    description='Subdimensão:',
    style={'description_width': 'initial'}
)

def atualizar_subdimensoes(*args):
    subdimensoes = df_todas_devolutivas[
        df_todas_devolutivas['dimensao'] == dropdown_dimensao.value
    ]['subdimensao'].unique().tolist()
    dropdown_subdimensao.options = subdimensoes
    if subdimensoes:
        dropdown_subdimensao.value = subdimensoes[0]

dropdown_dimensao.observe(atualizar_subdimensoes, names='value')
atualizar_subdimensoes()

# === SLIDER DE PONTUAÇÃO ===
entrada_pontuacao = widgets.IntSlider(
    value=17,
    min=0,
    max=51,
    step=1,
    description='Pontuação:',
    style={'description_width': 'initial'},
    continuous_update=False
)

# === BOTÃO E ÁREA DE OUTPUT ===
botao_gerar = widgets.Button(description='Gerar devolutiva', button_style='success')
output = widgets.Output()

# === FUNÇÃO DE GERAÇÃO FINAL ===
def ao_clicar_botao(b):
    with output:
        output.clear_output()

        # === Gera texto da devolutiva ===
        texto = gerar_devolutiva_dimensao(
            pontuacao=entrada_pontuacao.value,
            dimensao=dropdown_dimensao.value,
            subdimensao=dropdown_subdimensao.value,
            df=df_todas_devolutivas
        )

        display(Markdown("## 📄 **Devolutiva personalizada:**"))
        display(Markdown(texto))

        if "❌" in texto:
            display(Markdown("⚠️ Não foi possível gerar recomendações."))
            return

        # === Embedding e busca nos materiais ===
        embedding = gerar_embedding_para_rag(texto)
        distancias, indices = index.search(np.array(embedding).astype("float32"), 5)
        resultados = df_odas.iloc[indices[0]].copy()
        resultados["distância"] = distancias[0]

        # === Apresentação final ===
        markdown = "## 📚 **Materiais recomendados com base na sua devolutiva:**\n\n"
        for i, row in resultados.iterrows():
            titulo = row.get("Título", "Material sem título")
            resumo = row.get("Resumo", "Sem resumo disponível.").strip()
            suporte = row.get("Suporte", "Não informado")
            dimensao = row.get("Dimensões", "Não informado")
            duracao = row.get("Descricao_duracao", "⏱️ Duração não informada")
            link_real = row.get("Fonte", "#").strip()

            markdown += f"""
**{i+1}. [{titulo}]({link_real})**
- 📝 **Resumo:** {resumo}
- 📎 **Tipo:** {suporte} | **Dimensão:** {dimensao}
- ⏱️ **Duração:** {duracao}
- 📏 **Similaridade:** {row['distância']:.4f}

---
"""

        display(Markdown(markdown))

botao_gerar.on_click(ao_clicar_botao)

# === EXIBIR A INTERFACE ===
display(dropdown_dimensao, dropdown_subdimensao, entrada_pontuacao, botao_gerar, output)

Dropdown(description='Dimensão:', options=('pedag', 'pessoal'), style=DescriptionStyle(description_width='init…

Dropdown(description='Subdimensão:', options=('monitoriamento', 'desenvolvimento', 'implementacao', 'protecao'…

IntSlider(value=17, continuous_update=False, description='Pontuação:', max=51, style=SliderStyle(description_w…

Button(button_style='success', description='Gerar devolutiva', style=ButtonStyle())

Output()

In [ ]:
# prompt: salvar em csv

# Supondo que o DataFrame df_todas_devolutivas já esteja carregado no ambiente
# Vamos salvar o arquivo em formato CSV com separador ';' e codificação UTF-8
caminho_csv_exemplo = "/content/drive/MyDrive/exemplo_devolutivas_padronizadas.csv"  # Use um caminho válido no seu Google Drive
df_todas_devolutivas.to_csv(caminho_csv_exemplo, sep=";", encoding="utf-8", index=False)

print(f"DataFrame salvo em: {caminho_csv_exemplo}")


DataFrame salvo em: /content/drive/MyDrive/exemplo_devolutivas_padronizadas.csv


## LLM

In [ ]:
!pip install --quiet tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.0 MB/s eta 0:00:00


In [ ]:
import openai
from openai import OpenAI
import tiktoken
import os

# Configurar a API
client = OpenAI(api_key=)  # ou use os.environ["OPENAI_API_KEY"]

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import faiss
import pickle
import openai

# Carregar modelo e dados
modelo = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.read_index("odas_index.faiss")
with open("metadados_odas.pkl", "rb") as f:
    metadados = pickle.load(f)

# Texto da devolutiva (exemplo)
texto_devolutiva = "🔢 Pontuação: 20 ... (coloque aqui o texto gerado)"

# Gerar embedding e buscar os 25 mais similares
embedding = modelo.encode([texto_devolutiva])
distancias, indices = index.search(np.array(embedding).astype("float32"), 25)
materiais = metadados.iloc[indices[0]].copy()
materiais["distância"] = distancias[0]

# Criar texto para LLM
trecho = texto_devolutiva.split("**Necessidades formativas:**")[-1].strip()
topicos = "\n".join([f"- {row['Link fixo']} ({row['Descricao_duracao']})" for _, row in materiais.iterrows()])

# Criar o prompt
prompt = f"""
Você é um especialista em educação e recomendação de materiais. Abaixo está o trecho de uma devolutiva educacional com tópicos de formação que a pessoa precisa desenvolver:

TEXTO BASE:
{trecho}

A seguir estão 25 materiais que podem ser úteis. Analise quais são os mais relevantes com base no conteúdo acima e classifique por tipo:

{topicos}

Para cada grupo abaixo, selecione os materiais apropriados:
1. 🎥 Vídeos curtos (ideais para introdução rápida)
2. 📘 Leituras médias (para aprofundamento moderado)
3. 📚 Leituras longas (para estudo aprofundado)
4. 📌 Outros recursos adicionais (se for o caso)

Escreva sua recomendação de forma clara e organizada, explicando como cada grupo pode ajudar a desenvolver as competências desejadas.
"""

# Fazer a chamada à LLM com a API nova
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Você é um assistente educacional especializado em recomendação de materiais."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

# Mostrar resposta
print(response.choices[0].message.content)

Como a devolutiva educacional e os tópicos de formação específicos não foram fornecidos, vou classificar os materiais de acordo com o tipo e potencial de utilidade genérica. 

1. 🎥 Vídeos curtos (ideais para introdução rápida):
   - desafios-de-estudar-na-pandemia-gestao-da-educacao-publica-em-tempos-de-crise-instituto-unibanco
   - tr-educacao-conectada-webinar-1-como-preparar-uma-boa-aula-no-ensino-remoto-de-emergencia
   - webinario-o-desafio-da-volta-as-aulas-contribuicoes-para-o-debate-publico
   - ta-politicas-de-fortalecimento-da-gestao-escolar-instituto-unibanco-educacao-para-juventudes
   - ta-voluntariado-na-educacao
   
Estes vídeos podem fornecer uma visão geral rápida dos temas abordados e introduzir as principais ideias.

2. 📘 Leituras médias (para aprofundamento moderado):
   - tf-covid-19-nada-sera-como-antes
   - ta-educacao-360-2o-dia-se-nao-olharmos-para-o-historico-essa-educacao-sem-sentido-nao-conseguiremos-avancar
   - tf-educacao-e-covid-19-cenario-para-o-avanco-

In [ ]:
def contar_tokens(texto, modelo="gpt-4"):
    import tiktoken
    encoding = tiktoken.encoding_for_model(modelo)
    return len(encoding.encode(texto))

def gerar_recomendacao_llm(texto_devolutiva: str, df_materiais, modelo="gpt-4", max_tokens_total=8192):
    # 1. Criar embedding da devolutiva
    embedding = gerar_embedding_para_rag(texto_devolutiva)
    distancias, indices = index.search(np.array(embedding).astype("float32"), 100)  # busca os 100 mais próximos
    materiais = df_materiais.iloc[indices[0]].copy()
    materiais["distância"] = distancias[0]

    # 2. Prompt base + devolutiva
    prompt_base = f"""
Você é um assistente educacional. Recebeu uma devolutiva de avaliação formativa e uma lista de materiais. Seu papel é:

1. Classificar os materiais em categorias:
   - 🎥 Vídeos introdutórios
   - 📘 Leituras médias
   - 📚 Leituras longas
   - 🧩 Recursos adicionais

2. Explicar para o usuário como cada categoria pode ser útil no processo formativo, e quando é melhor usar vídeos curtos, artigos longos etc.

### Devolutiva:
{texto_devolutiva}

### Materiais:"""

    tokens_usados = contar_tokens(prompt_base, modelo=modelo)
    token_limite_materiais = max_tokens_total - tokens_usados - 1000  # margem para resposta

    # 3. Montar bloco de materiais dentro do limite
    materiais_texto = ""
    total_tokens = 0
    for _, row in materiais.iterrows():
        bloco = f"""
Título: {row.get('Link fixo')}
Resumo: {row.get('Resumo', 'Sem resumo.').strip()}
Tipo: {row.get('Suporte')}
Duração: {row.get('Descricao_duracao')}
Link: {row.get('Fonte')}
"""
        blocos_tokens = contar_tokens(bloco, modelo=modelo)
        if total_tokens + blocos_tokens > token_limite_materiais:
            break
        materiais_texto += bloco
        total_tokens += blocos_tokens

    # 4. Chamada à API da OpenAI
    prompt_final = prompt_base + "\n\n" + materiais_texto

    resposta = client.chat.completions.create(
        model=modelo,
        messages=[{"role": "user", "content": prompt_final}],
        temperature=0.7
    )

    return resposta.choices[0].message.content.strip()

In [ ]:
recomendacao_final = gerar_recomendacao_llm(texto, df_odas)
print(recomendacao_final)

### Classificação dos materiais:

#### 🎥 Vídeos introdutórios
Não há vídeos na lista de materiais.

#### 📘 Leituras médias
1. [O papel da coordenação pedagógica na regulação das avaliações de aprendizagem junto aos docentes](https://www.scielo.br/j/ensaio/a/Y9vSywR8xcrsVgzNT4cZfpd/?format=pdf&lang=pt) (24 páginas)
2. [Planejamento de sistema educacional a longo prazo: experiências internacionais e diálogos com o Brasil](https://d3e.com.br/wp-content/uploads/nota-tecnica_2401_planejamento-sistema-educacional-longo-prazo.pdf) (16 páginas)
3. [Práticas de gestão, liderança educativa e qualidade da educação em escolas de ensino médio no Brasil](https://www.scielo.br/j/ensaio/a/Y9vSywR8xcrsVgzNT4cZfpd/?format=pdf&lang=pt) (31 páginas)
4. [Orientações governamentais para a prática da avaliação de aprendizagem no cenário pandêmico de COVID-19 no estado do Paraná](https://www.scielo.br/j/ensaio/a/Y9vSywR8xcrsVgzNT4cZfpd/?format=pdf&lang=pt) (23 páginas)
5. [As práticas letivas na educação físi

In [ ]:
# === CRIAÇÃO DA COLUNA Descricao_duracao ===
def classificar_duracao(valor):
    if pd.isna(valor):
        return "⏱️ Duração não informada"
    texto = str(valor).lower()
    if "hora" in texto or ":" in texto:
        return f"🎥 {valor} (vídeo)"
    elif "página" in texto or texto.isdigit():
        return f"📚 {valor} (artigo)"
    return f"⏱️ {valor}"

if "Descricao_duracao" not in df_odas.columns:
    df_odas["Descricao_duracao"] = df_odas["Duração/Tamanho"].apply(classificar_duracao)

# === EMBEDDING DO TEXTO PARA CONSULTA ===
def gerar_embedding_para_rag(texto: str) -> np.ndarray:
    if "**Necessidades formativas:**" in texto:
        trecho = texto.split("**Necessidades formativas:**")[-1].strip()
    else:
        trecho = texto
    return modelo.encode([trecho])

# === FORMATAÇÃO E CHAMADA LLM ===
def gerar_recomendacao_llm(texto_devolutiva: str, materiais: pd.DataFrame, top_n: int = 25):
    materiais = materiais.sort_values("distância").head(top_n)

    lista = []
    for i, row in materiais.iterrows():
        bloco = f"""
Título: {row['Link fixo']}
Resumo: {row['Resumo']}
Tipo: {row['Suporte']}
Duração: {row['Descricao_duracao']}
Link: {row['Fonte']}
"""
        lista.append(bloco)

    prompt = f"""
Você é um especialista em educação. Com base na devolutiva abaixo e nos materiais listados, selecione e organize os conteúdos recomendados por tipo (vídeos curtos, leituras médias, artigos longos etc). Explique como cada tipo ajuda o gestor a se desenvolver. Use linguagem clara e objetiva, e formate com subtítulos e listas numeradas.

### Devolutiva:
{texto_devolutiva}

### Materiais disponíveis:
{''.join(lista)}
"""

    resposta = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
    )

    return resposta.choices[0].message.content.strip()

# === DROPDOWNS INTERATIVOS ===
dimensoes_unicas = df_todas_devolutivas['dimensao'].unique().tolist()

dropdown_dimensao = widgets.Dropdown(
    options=dimensoes_unicas,
    description='Dimensão:',
    value=dimensoes_unicas[0],
    style={'description_width': 'initial'}
)

dropdown_subdimensao = widgets.Dropdown(
    options=[],
    description='Subdimensão:',
    style={'description_width': 'initial'}
)

def atualizar_subdimensoes(*args):
    subdimensoes = df_todas_devolutivas[
        df_todas_devolutivas['dimensao'] == dropdown_dimensao.value
    ]['subdimensao'].unique().tolist()
    dropdown_subdimensao.options = subdimensoes
    if subdimensoes:
        dropdown_subdimensao.value = subdimensoes[0]

dropdown_dimensao.observe(atualizar_subdimensoes, names='value')
atualizar_subdimensoes()

# === SLIDER DE PONTUAÇÃO ===
entrada_pontuacao = widgets.IntSlider(
    value=17,
    min=0,
    max=51,
    step=1,
    description='Pontuação:',
    style={'description_width': 'initial'},
    continuous_update=False
)

# === BOTÃO E OUTPUT ===
botao_gerar = widgets.Button(description='Gerar devolutiva e recomendações', button_style='success')
output = widgets.Output()

def ao_clicar_botao(b):
    with output:
        output.clear_output()

        texto = gerar_devolutiva_dimensao(
            pontuacao=entrada_pontuacao.value,
            dimensao=dropdown_dimensao.value,
            subdimensao=dropdown_subdimensao.value,
            df=df_todas_devolutivas
        )

        display(Markdown("## 📄 **Devolutiva personalizada:**"))
        display(Markdown(texto))

        if "❌" in texto:
            display(Markdown("⚠️ Não foi possível gerar recomendações."))
            return

        embedding = gerar_embedding_para_rag(texto)
        distancias, indices = index.search(np.array(embedding).astype("float32"), 50)
        resultados = df_odas.iloc[indices[0]].copy()
        resultados["distância"] = distancias[0]

        # CHAMADA DA LLM COM OS TOP 25
        recomendacao_final = gerar_recomendacao_llm(texto, resultados)
        display(Markdown("## 📚 **Materiais recomendados com base na sua devolutiva:**"))
        display(Markdown(recomendacao_final))

botao_gerar.on_click(ao_clicar_botao)

# === EXIBIR A INTERFACE ===
display(dropdown_dimensao, dropdown_subdimensao, entrada_pontuacao, botao_gerar, output)

Dropdown(description='Dimensão:', options=('pedag', 'pessoal'), style=DescriptionStyle(description_width='init…

Dropdown(description='Subdimensão:', options=('monitoriamento', 'desenvolvimento', 'implementacao', 'protecao'…

IntSlider(value=17, continuous_update=False, description='Pontuação:', max=51, style=SliderStyle(description_w…

Button(button_style='success', description='Gerar devolutiva e recomendações', style=ButtonStyle())

Output()

In [ ]:
!pip freeze | grep -E 'faiss|torch|transformers|sentence-transformers|scikit-learn|numpy'

faiss-cpu==1.10.0
numpy==2.0.2
scikit-learn==1.6.1
sentence-transformers==3.4.1
torch @ https://download.pytorch.org/whl/cu124/torch-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchaudio @ https://download.pytorch.org/whl/cu124/torchaudio-2.6.0%2Bcu124-cp311-cp311-linux_x86_64.whl
torchsummary==1.5.1
torchvision @ https://download.pytorch.org/whl/cu124/torchvision-0.21.0%2Bcu124-cp311-cp311-linux_x86_64.whl
transformers==4.50.3


## e5-large-v2

In [ ]:
from sentence_transformers import SentenceTransformer

# Carregar modelo robusto com melhor desempenho e compatível com GPU no Colab
# Neste caso: intfloat/e5-large-v2
modelo_robusto = SentenceTransformer("intfloat/e5-large-v2")
modelo_robusto.cuda()  # Envia para GPU, se disponível

# Confirmação
modelo_robusto.device

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

device(type='cuda', index=0)

In [ ]:
# Reimportar e recarregar após reset do ambiente

from sentence_transformers import SentenceTransformer
import torch

# Carregar modelo robusto e enviar para GPU (se disponível)
modelo_robusto = SentenceTransformer("intfloat/e5-large-v2")
modelo_robusto.cuda()

# Mostrar em qual dispositivo está o modelo
modelo_robusto.device

device(type='cuda', index=0)

In [ ]:
# === IMPORTAÇÕES ===
import pandas as pd
import numpy as np
import faiss
import pickle
import re
from sentence_transformers import SentenceTransformer
import torch

# === 1. Carregar os dados do repositório ===
caminho_csv = "/content/drive/MyDrive/rag-gestores/data/odas/repositorio-odas.csv"
df_odas = pd.read_csv(caminho_csv, sep=";", encoding="utf-8")

# === 2. Interpretar a coluna Duração/Tamanho ===
def interpretar_duracao(duracao):
    if pd.isna(duracao) or duracao.strip().lower() in ['s/d', '']:
        return "⏱️ Duração não informada"
    if "min" in duracao:
        minutos = int(re.findall(r"\d+", duracao)[0])
        if minutos <= 5:
            return f"🎥 {duracao} (vídeo curto)"
        elif minutos <= 20:
            return f"🎬 {duracao} (vídeo médio)"
        else:
            return f"🧑‍🏫 {duracao} (aula ou curso)"
    elif "página" in duracao:
        paginas = int(re.findall(r"\d+", duracao)[0])
        if paginas <= 3:
            return f"📄 {duracao} (texto rápido)"
        elif paginas <= 10:
            return f"📘 {duracao} (leitura média)"
        else:
            return f"📚 {duracao} (artigo extenso)"
    return f"📌 {duracao}"

df_odas["Descricao_duracao"] = df_odas["Duração/Tamanho"].apply(interpretar_duracao)

# === 3. Criar campo de texto completo para embeddings ===
def preparar_texto_oda(row):
    partes = [
        f"Título: {row.get('Título', '')}",
        f"Resumo: {row.get('Resumo', '')}",
        f"Temas: {row.get('Temas', '')}",
        f"Palavras-chave: {row.get('Palavras-chave - Termos do vocabulário controlado', '')}",
        f"Dimensões: {row.get('Dimensões', '')}",
        f"Suporte: {row.get('Suporte', '')}",
        f"Duração: {row.get('Descricao_duracao', '')}",
    ]
    return "\n".join([p for p in partes if pd.notnull(p)])

df_odas["texto_completo"] = df_odas.apply(preparar_texto_oda, axis=1)

# === 4. Gerar embeddings com modelo robusto na GPU ===
modelo = SentenceTransformer("intfloat/e5-large-v2")
modelo = modelo.cuda()
embeddings = modelo.encode(
    df_odas["texto_completo"].tolist(),
    show_progress_bar=True,
    device="cuda"
)

# === 5. Criar índice FAISS ===
dimensao = embeddings.shape[1]
index = faiss.IndexFlatL2(dimensao)
index.add(np.array(embeddings).astype("float32"))

# === 6. Guardar metadados úteis para exibição futura ===
metadados = df_odas[[
    "Título",
    "Link fixo",
    "Fonte",
    "Resumo",
    "Suporte",
    "Dimensões",
    "Descricao_duracao",
    "texto_completo"
]].reset_index(drop=True)

# === 7. Salvar o índice e metadados ===
with open("metadados_odas.pkl", "wb") as f:
    pickle.dump(metadados, f)
faiss.write_index(index, "odas_index.faiss")

# === 8. Verificação final ===
print("Index size:", index.ntotal)
print(metadados.head(3))


Batches:   0%|          | 0/76 [00:00<?, ?it/s]

Index size: 2422
                                              Título  \
0  A qualidade da educação no horizonte da proteç...   
1  Relatório de pesquisa – Papel da Supervisora E...   
2  Relatório de pesquisa – Avaliação responsiva e...   

                                           Link fixo  \
0  a-qualidade-da-educacao-no-horizonte-da-protec...   
1  relatorio-de-pesquisa-papel-da-supervisora-esc...   
2  relatorio-de-pesquisa-avaliacao-responsiva-em-...   

                                               Fonte  \
0  https://www.scielo.br/j/cp/a/3gRPNHGZhvgfYGkYf...   
1                                                      
2                                                      

                                              Resumo  \
0  <p>O artigo pretende problematizar o conceito ...   
1  Esta pesquisa, tem como finalidade investigar ...   
2  O presente estudo de avaliação responsiva pret...   

                                Suporte  \
0  9293d806-8419-4127-b442-6437549105e5 

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import numpy as np
import pandas as pd

# === CRIAÇÃO DA COLUNA Descricao_duracao ===
def classificar_duracao(valor):
    if pd.isna(valor):
        return "⏱️ Duração não informada"
    texto = str(valor).lower()
    if "hora" in texto or ":" in texto:
        return f"🎥 {valor} (vídeo)"
    elif "página" in texto or texto.isdigit():
        return f"📚 {valor} (artigo)"
    return f"⏱️ {valor}"

if "Descricao_duracao" not in df_odas.columns:
    df_odas["Descricao_duracao"] = df_odas["Duração/Tamanho"].apply(classificar_duracao)

# === FUNÇÃO PARA EMBEDDING A PARTIR DA DEVOLUTIVA ===
def gerar_embedding_para_rag(texto: str) -> np.ndarray:
    # Tenta extrair parte mais relevante para recomendação
    trecho = ""
    if "**Necessidades formativas:**" in texto:
        trecho = texto.split("**Necessidades formativas:**")[-1].strip()
    elif "**Seus pontos fortes:**" in texto:
        trecho = texto.split("**Seus pontos fortes:**")[-1].strip()
    else:
        trecho = texto  # fallback
    return modelo.encode([trecho])

# === DROPDOWNS INTERATIVOS ===
dimensoes_unicas = df_todas_devolutivas['dimensao'].unique().tolist()

dropdown_dimensao = widgets.Dropdown(
    options=dimensoes_unicas,
    description='Dimensão:',
    value=dimensoes_unicas[0],
    style={'description_width': 'initial'}
)

dropdown_subdimensao = widgets.Dropdown(
    options=[],
    description='Subdimensão:',
    style={'description_width': 'initial'}
)

def atualizar_subdimensoes(*args):
    subdimensoes = df_todas_devolutivas[
        df_todas_devolutivas['dimensao'] == dropdown_dimensao.value
    ]['subdimensao'].unique().tolist()
    dropdown_subdimensao.options = subdimensoes
    if subdimensoes:
        dropdown_subdimensao.value = subdimensoes[0]

dropdown_dimensao.observe(atualizar_subdimensoes, names='value')
atualizar_subdimensoes()

# === SLIDER DE PONTUAÇÃO ===
entrada_pontuacao = widgets.IntSlider(
    value=17,
    min=0,
    max=51,
    step=1,
    description='Pontuação:',
    style={'description_width': 'initial'},
    continuous_update=False
)

# === BOTÃO E ÁREA DE OUTPUT ===
botao_gerar = widgets.Button(description='Gerar devolutiva', button_style='success')
output = widgets.Output()

# === FUNÇÃO DE GERAÇÃO FINAL ===
def ao_clicar_botao(b):
    with output:
        output.clear_output()

        # === Gera texto da devolutiva ===
        texto = gerar_devolutiva_dimensao(
            pontuacao=entrada_pontuacao.value,
            dimensao=dropdown_dimensao.value,
            subdimensao=dropdown_subdimensao.value,
            df=df_todas_devolutivas
        )

        display(Markdown("## 📄 **Devolutiva personalizada:**"))
        display(Markdown(texto))

        if "❌" in texto:
            display(Markdown("⚠️ Não foi possível gerar recomendações."))
            return

        # === Embedding e busca nos materiais ===
        embedding = gerar_embedding_para_rag(texto)
        distancias, indices = index.search(np.array(embedding).astype("float32"), 5)
        resultados = df_odas.iloc[indices[0]].copy()
        resultados["distância"] = distancias[0]

        # === Apresentação final ===
        markdown = "## 📚 **Materiais recomendados com base na sua devolutiva:**\n\n"
        for i, row in resultados.iterrows():
            titulo = row.get("Título", "Material sem título")
            resumo = row.get("Resumo", "Sem resumo disponível.").strip()
            suporte = row.get("Suporte", "Não informado")
            dimensao = row.get("Dimensões", "Não informado")
            duracao = row.get("Descricao_duracao", "⏱️ Duração não informada")
            link_real = row.get("Fonte", "#").strip()

            markdown += f"""
**{i+1}. [{titulo}]({link_real})**
- 📝 **Resumo:** {resumo}
- 📎 **Tipo:** {suporte} | **Dimensão:** {dimensao}
- ⏱️ **Duração:** {duracao}
- 📏 **Similaridade:** {row['distância']:.4f}

---
"""

        display(Markdown(markdown))

botao_gerar.on_click(ao_clicar_botao)

# === EXIBIR A INTERFACE ===
display(dropdown_dimensao, dropdown_subdimensao, entrada_pontuacao, botao_gerar, output)

NameError: name 'df_todas_devolutivas' is not defined

## Stella v5

In [ ]:
import pandas as pd
import numpy as np
import faiss
import pickle
import re
from sentence_transformers import SentenceTransformer
import gc
import torch
from sklearn.preprocessing import normalize

# === Limpeza de memória ===
gc.collect()
torch.cuda.empty_cache()

# === 1. Carregar os dados do repositório ===
caminho_csv = "/content/drive/MyDrive/rag-gestores/data/odas/repositorio-odas.csv"
df_odas = pd.read_csv(caminho_csv, sep=";", encoding="utf-8")

# === 2. Interpretar a coluna Duração/Tamanho ===
def interpretar_duracao(duracao):
    if pd.isna(duracao) or duracao.strip().lower() in ['s/d', '']:
        return "⏱️ Duração não informada"
    if "min" in duracao:
        minutos = int(re.findall(r"\d+", duracao)[0])
        if minutos <= 5:
            return f"🎥 {duracao} (vídeo curto)"
        elif minutos <= 20:
            return f"🎬 {duracao} (vídeo médio)"
        else:
            return f"🧑‍🏫 {duracao} (aula ou curso)"
    elif "página" in duracao:
        paginas = int(re.findall(r"\d+", duracao)[0])
        if paginas <= 3:
            return f"📄 {duracao} (texto rápido)"
        elif paginas <= 10:
            return f"📘 {duracao} (leitura média)"
        else:
            return f"📚 {duracao} (artigo extenso)"
    return f"📌 {duracao}"

df_odas["Descricao_duracao"] = df_odas["Duração/Tamanho"].apply(interpretar_duracao)

# === 3. Criar campo de texto completo para embeddings ===
def preparar_texto_oda(row):
    partes = [
        f"Título: {row.get('Título', '')}",
        f"Resumo: {row.get('Resumo', '')}",
        f"Temas: {row.get('Temas', '')}",
        f"Palavras-chave: {row.get('Palavras-chave - Termos do vocabulário controlado', '')}",
        f"Dimensões: {row.get('Dimensões', '')}",
        f"Suporte: {row.get('Suporte', '')}",
        f"Duração: {row.get('Descricao_duracao', '')}",
    ]
    return "\n".join([p for p in partes if pd.notnull(p)])

df_odas["texto_completo"] = df_odas.apply(preparar_texto_oda, axis=1)

# === 4. Gerar embeddings com modelo Stella v1.5 ===
modelo = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
modelo = modelo.cuda()

embeddings = modelo.encode(
    df_odas["texto_completo"].tolist(),
    batch_size=8,  # ajuste conforme limite de memória
    show_progress_bar=True,
    convert_to_numpy=True
)

# === 5. Normalizar embeddings para usar com FAISS + IndexFlatIP ===
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# === 6. Criar índice FAISS com produto interno (cosine similarity)
dimensao = embeddings.shape[1]
index = faiss.IndexFlatIP(dimensao)
index.add(embeddings.astype("float32"))

# === 7. Guardar metadados úteis para exibição futura ===
metadados = df_odas[[
    "Título",
    "Link fixo",
    "Fonte",
    "Resumo",
    "Suporte",
    "Dimensões",
    "Descricao_duracao",
    "texto_completo"
]].reset_index(drop=True)

# === 8. Salvar índice e metadados ===
with open("metadados_odas.pkl", "wb") as f:
    pickle.dump(metadados, f)

faiss.write_index(index, "odas_index.faiss")

# === 9. Verificação final ===
print("Index size:", index.ntotal)
print(metadados.head(3))

Batches:   0%|          | 0/283 [00:00<?, ?it/s]

✅ Index criado com 2262 documentos.
                                              Título  \
0  Currículo Estadual de São Paulo – Etapa Ensino...   
1  Currículo do Espírito Santo – Texto introdutór...   
2  Currículo do Espírito Santo – Ciências Humanas...   

                                           Link fixo  \
0            a-curriculo-paulista-etapa-ensino-medio   
1  a-curriculo-do-espirito-santo-texto-introdutor...   
2  a-curriculo-do-espirito-santo-ciencias-humanas...   

                                               Fonte  \
0  https://efape.educacao.sp.gov.br/curriculopaul...   
1  https://drive.google.com/file/d/1B41xYpgWgvZGd...   
2  https://drive.google.com/file/d/1U908DVKACU5vs...   

                                              Resumo Suporte  \
0  <p>O curr&iacute;culo escolar do Ensino M&eacu...   Texto   
1  <p>O Curr&iacute;culo escolar do Ensino M&eacu...   Texto   
2  <p>O Curr&iacute;culo escolar do Ensino M&eacu...   Texto   

                                

In [ ]:
# === IMPORTAÇÕES ===
import ipywidgets as widgets
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
import faiss
import pickle
import re
from sentence_transformers import SentenceTransformer

# === 1. CARREGAR MODELO, METADADOS E ÍNDICE ===
with open("metadados_odas.pkl", "rb") as f:
    df_odas = pickle.load(f)

index = faiss.read_index("odas_index.faiss")

modelo = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
modelo = modelo.cuda()

# === 2. CARREGAR DEVOLUTIVAS E RUBRICAS ===
df_devolutivas = pd.read_csv("Devolutivas.csv", sep=";")
df_rubricas = pd.read_csv("Rubricas.csv", sep=";")

# Corrigir nome de coluna se necessário
df_devolutivas = df_devolutivas.rename(columns={"Necessidaes formativas": "Necessidades formativas"})

# === 3. FUNÇÕES DE APOIO ===
def classificar_duracao(valor):
    if pd.isna(valor):
        return "⏱️ Duração não informada"
    texto = str(valor).lower()
    if "hora" in texto or ":" in texto:
        return f"🎥 {valor} (vídeo)"
    elif "página" in texto or texto.isdigit():
        return f"📚 {valor} (artigo)"
    return f"⏱️ {valor}"

if "Descricao_duracao" not in df_odas.columns:
    df_odas["Descricao_duracao"] = df_odas["Duração/Tamanho"].apply(classificar_duracao)

def encontrar_rubrica(pontuacao, dimensao, subdimensao):
    candidatos = df_rubricas[
        (df_rubricas['dimensao'] == dimensao) &
        (df_rubricas['subdimensao'] == subdimensao) &
        (df_rubricas['faixa_total_min'] <= pontuacao) &
        (df_rubricas['faixa_total_max'] >= pontuacao)
    ]
    if candidatos.empty:
        return None, None, None

    rubrica_numero = candidatos.iloc[0]['rubrica_numero']
    rubrica_nome = candidatos.iloc[0]['rubrica_nome']

    faixa_escolhida = candidatos[
        (candidatos['subfaixa_min'] <= pontuacao) &
        (candidatos['subfaixa_max'] >= pontuacao)
    ]
    if faixa_escolhida.empty:
        return rubrica_numero, rubrica_nome, None

    tipo_faixa = faixa_escolhida.iloc[0]['tipo_faixa']
    return rubrica_numero, rubrica_nome, tipo_faixa

def gerar_texto_devolutiva_rico(pontuacao, dimensao, subdimensao):
    rubrica_numero, rubrica_nome, tipo_faixa = encontrar_rubrica(pontuacao, dimensao, subdimensao)

    if not rubrica_numero or not tipo_faixa:
        return None

    rubrica_nome_completo = f"{rubrica_nome} – Nível {tipo_faixa}"

    devolutiva = df_devolutivas[
        (df_devolutivas['Dimensão'] == dimensao) &
        (df_devolutivas['Subdimensão'] == subdimensao) &
        (df_devolutivas['Rubrica numero'] == rubrica_numero) &
        (df_devolutivas['Rubrica nome'] == rubrica_nome_completo)
    ]

    if devolutiva.empty:
        return None

    item = devolutiva.iloc[0]

    texto = f"""
Dimensão: {dimensao}
Subdimensão: {subdimensao}
Rubrica: Rubrica {rubrica_numero} - {rubrica_nome}
Nível: {tipo_faixa}

Seus pontos fortes:
{item['Pontos fortes']}

O que fazer para avançar:
{item['O que fazer para avançar']}

Necessidades formativas:
{item['Necessidades formativas']}
""".strip()

    return texto

def gerar_embedding_para_rag(texto: str) -> np.ndarray:
    embedding = modelo.encode([texto])
    embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)
    return embedding

def interpretar_similaridade(valor):
    if valor >= 0.80:
        return "🔥 Altamente relevante"
    elif valor >= 0.65:
        return "✅ Relevante"
    elif valor >= 0.50:
        return "🧐 Moderadamente relevante"
    else:
        return "🔍 Pouco relevante"

def gerar_card_material(row, i):
    titulo = row.get("Título", "Sem título")
    resumo = re.sub(r"<[^>]+>", "", str(row.get("Resumo", "Sem resumo disponível")).strip())
    suporte = row.get("Suporte", "Não informado")
    dimensao = row.get("Dimensões", "Não informado")
    duracao = row.get("Descricao_duracao", "⏱️ Duração não informada")
    link_real = str(row.get("Fonte", "#")).strip()
    if link_real.lower() == "nan" or link_real == "":
        link_real = "#"
    sim = float(row['distância'])
    interpretacao = interpretar_similaridade(sim)

    return f"""
**{i+1}. [{titulo}]({link_real})**
- 📝 **Resumo:** {resumo}
- 📎 **Tipo:** {suporte} | **Dimensão:** {dimensao}
- ⏱️ **Duração:** {duracao}
- 📏 **Similaridade:** {sim:.4f} – *{interpretacao}*

---
"""

# === 4. INTERFACE COM IPYWIDGETS ===
dimensoes_unicas = df_devolutivas['Dimensão'].unique().tolist()

dropdown_dimensao = widgets.Dropdown(
    options=dimensoes_unicas,
    description='Dimensão:',
    value=dimensoes_unicas[0],
    style={'description_width': 'initial'}
)

dropdown_subdimensao = widgets.Dropdown(
    options=[],
    description='Subdimensão:',
    style={'description_width': 'initial'}
)

def atualizar_subdimensoes(*args):
    subdimensoes = df_devolutivas[
        df_devolutivas['Dimensão'] == dropdown_dimensao.value
    ]['Subdimensão'].unique().tolist()
    dropdown_subdimensao.options = subdimensoes
    if subdimensoes:
        dropdown_subdimensao.value = subdimensoes[0]

dropdown_dimensao.observe(atualizar_subdimensoes, names='value')
atualizar_subdimensoes()

entrada_pontuacao = widgets.IntSlider(
    value=17,
    min=0,
    max=51,
    step=1,
    description='Pontuação:',
    style={'description_width': 'initial'},
    continuous_update=False
)

botao_gerar = widgets.Button(description='Gerar devolutiva', button_style='success')
output = widgets.Output()

def ao_clicar_botao(b):
    with output:
        output.clear_output()

        texto = gerar_texto_devolutiva_rico(
            pontuacao=entrada_pontuacao.value,
            dimensao=dropdown_dimensao.value,
            subdimensao=dropdown_subdimensao.value
        )

        if texto is None:
            display(Markdown("⚠️ Não foi possível gerar devolutiva para essa pontuação."))
            return

        display(Markdown("## 📄 **Devolutiva personalizada:**"))
        display(Markdown(texto))

        embedding = gerar_embedding_para_rag(texto)
        distancias, indices = index.search(np.array(embedding).astype("float32"), 50)
        resultados = df_odas.iloc[indices[0]].copy()
        resultados["distância"] = distancias[0]

        # Filtro: apenas materiais em português
        resultados = resultados[resultados["Idiomas"].str.contains("português", case=False, na=False)]

        # Organizar artigos e vídeos
        artigos = resultados[resultados["Suporte"].str.contains("Artigo|Texto|Livro", case=False, na=False)]
        videos = resultados[resultados["Suporte"].str.contains("Vídeo|Aula|Curso", case=False, na=False)]

        markdown = "## 📚 **Materiais recomendados – Artigos:**\n\n"
        for i, row in artigos.iterrows():
            markdown += gerar_card_material(row, i)

        markdown += "\n\n## 🎥 **Materiais recomendados – Vídeos:**\n\n"
        for i, row in videos.iterrows():
            markdown += gerar_card_material(row, i)

        display(Markdown(markdown))

botao_gerar.on_click(ao_clicar_botao)

display(dropdown_dimensao, dropdown_subdimensao, entrada_pontuacao, botao_gerar, output)

Dropdown(description='Dimensão:', options=('Dimensão pedagógica', 'Dimensão pessoal-relacional'), style=Descri…

Dropdown(description='Subdimensão:', options=('Planejamento pedagógico', 'Implementação do processo de ensino …

IntSlider(value=17, continuous_update=False, description='Pontuação:', max=51, style=SliderStyle(description_w…

Button(button_style='success', description='Gerar devolutiva', style=ButtonStyle())

Output()

Novos Dados

In [ ]:
# === IMPORTAÇÕES ===
import pandas as pd
import numpy as np
import faiss
import pickle
import gc
import torch
import re
from sentence_transformers import SentenceTransformer

# === LIMPAR MEMÓRIA ===
gc.collect()
torch.cuda.empty_cache()

# === 1. CARREGAR DADOS ===
caminho_xlsx = "relatorio-odas.xlsx"
df_odas = pd.read_excel(caminho_xlsx)

# === 2. PRÉ-FILTRAGEM ===
# Manter apenas ODAs com Resumo, Temas e Dimensões preenchidos
df_odas = df_odas.dropna(subset=["Resumo", "Temas", "Dimensões", "Tipo"])

# === 3. INTERPRETAR DURAÇÃO ===
def interpretar_duracao(duracao, suporte=""):
    if pd.isna(duracao) or str(duracao).strip().lower() in ['s/d', '']:
        return "⏱️ Duração não informada"

    texto = str(duracao).lower()
    minutos_total = 0

    # Extrair horas, minutos e segundos
    match_h = re.search(r"(\d+)\s*hora", texto)
    match_m = re.search(r"(\d+)\s*minuto", texto)
    match_s = re.search(r"(\d+)\s*segundo", texto)

    if match_h:
        minutos_total += int(match_h.group(1)) * 60
    if match_m:
        minutos_total += int(match_m.group(1))
    if match_s:
        minutos_total += int(match_s.group(1)) // 60

    suporte = suporte.lower()

    if "vídeo" in suporte:
        if minutos_total <= 5:
            return f"🎥 {duracao} (vídeo curto)"
        elif minutos_total <= 20:
            return f"🎬 {duracao} (vídeo médio)"
        else:
            return f"🧑‍🏫 {duracao} (aula longa)"

    elif "áudio" in suporte:
        if minutos_total <= 5:
            return f"🎧 {duracao} (áudio curto)"
        elif minutos_total <= 20:
            return f"🎙️ {duracao} (áudio médio)"
        else:
            return f"🎚️ {duracao} (áudio longo)"

    elif "página" in texto:
        paginas = int(re.findall(r"\d+", texto)[0])
        if paginas <= 3:
            return f"📄 {duracao} (texto curto)"
        elif paginas <= 10:
            return f"📘 {duracao} (leitura média)"
        else:
            return f"📚 {duracao} (artigo longo)"

    return f"🖋️ {duracao}"

df_odas["Descricao_duracao"] = df_odas.apply(
    lambda row: interpretar_duracao(row["Duração/Tamanho"], row.get("Suporte", "")), axis=1
)

# === 4. PREPARAR TEXTO PARA EMBEDDING (ENRIQUECIDO) ===
def preparar_texto_oda(row):
    partes = [
        f"Título: {row.get('Título', '')}",
        f"Resumo: {row.get('Resumo', '')}",
        f"Temas: {row.get('Temas', '')}",
        f"Palavras-chave: {row.get('Palavras-chave - Termos do vocabulário controlado', '')}",
        f"Dimensões: {row.get('Dimensões', '')}",
        f"Seção do Observatório: {row.get('Seção do Observatório', '')}",
        f"Instituições: {row.get('Instituições', '')}",
        f"Idioma: {row.get('Idiomas', '')}",
        f"Tipo de Suporte: {row.get('Suporte', '')}",
        f"Duração: {row.get('Descricao_duracao', '')}"
    ]
    return "\n".join([p for p in partes if pd.notnull(p) and str(p).strip() != ""])

df_odas["texto_completo"] = df_odas.apply(preparar_texto_oda, axis=1)

# === 5. GERAR EMBEDDINGS ===
modelo = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
modelo = modelo.cuda()

embeddings = modelo.encode(
    df_odas["texto_completo"].tolist(),
    batch_size=8,
    show_progress_bar=True,
    convert_to_numpy=True
)

# === 6. NORMALIZAR EMBEDDINGS ===
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

# === 7. CRIAR ÍNDICE FAISS ===
dimensao = embeddings.shape[1]
index = faiss.IndexFlatIP(dimensao)
index.add(embeddings.astype("float32"))

# === 8. SALVAR METADADOS E ÍNDICE ===
metadados = df_odas[[
    "Título", "Link fixo", "Fonte", "Resumo", "Suporte", "Dimensões",
    "Descricao_duracao", "texto_completo", "Seção do Observatório",
    "Idiomas", "Instituições", "id", "Temas", "Tipo", "Palavras-chave - Termos do vocabulário controlado"
]].reset_index(drop=True)

with open("metadados_odas_stellav5.pkl", "wb") as f:
    pickle.dump(metadados, f)

faiss.write_index(index, "odas_index_stellav5.faiss")

# === 9. FINALIZAÇÃO ===
print(f"✅ Index criado com {index.ntotal} documentos.")
print(metadados.head(3))

Batches:   0%|          | 0/283 [00:00<?, ?it/s]

✅ Index criado com 2262 documentos.
                                              Título  \
0  Currículo Estadual de São Paulo – Etapa Ensino...   
1  Currículo do Espírito Santo – Texto introdutór...   
2  Currículo do Espírito Santo – Ciências Humanas...   

                                           Link fixo  \
0            a-curriculo-paulista-etapa-ensino-medio   
1  a-curriculo-do-espirito-santo-texto-introdutor...   
2  a-curriculo-do-espirito-santo-ciencias-humanas...   

                                               Fonte  \
0  https://efape.educacao.sp.gov.br/curriculopaul...   
1  https://drive.google.com/file/d/1B41xYpgWgvZGd...   
2  https://drive.google.com/file/d/1U908DVKACU5vs...   

                                              Resumo Suporte  \
0  <p>O curr&iacute;culo escolar do Ensino M&eacu...   Texto   
1  <p>O Curr&iacute;culo escolar do Ensino M&eacu...   Texto   
2  <p>O Curr&iacute;culo escolar do Ensino M&eacu...   Texto   

                                

In [ ]:
# === IMPORTAÇÕES ===
import ipywidgets as widgets
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
import faiss
import pickle
import re
import gc
import torch
from sentence_transformers import SentenceTransformer

# === 1. CARREGAR MODELO, METADADOS E ÍNDICE ===
gc.collect()
torch.cuda.empty_cache()

with open("metadados_odas.pkl", "rb") as f:
    df_odas = pickle.load(f)

index = faiss.read_index("odas_index.faiss")

modelo = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
modelo = modelo.cuda()

# === 2. CARREGAR DEVOLUTIVAS E RUBRICAS ===
df_devolutivas = pd.read_csv("devolutivas.csv", sep=";")
df_rubricas = pd.read_csv("rubricas.csv", sep=";")

df_devolutivas = df_devolutivas.rename(columns={"Necessidaes formativas": "Necessidades formativas"})

# === 3. FUNÇÕES DE APOIO ===
def classificar_duracao(valor):
    if pd.isna(valor):
        return "⏱️ Duração não informada"
    texto = str(valor).lower()
    if "hora" in texto or ":" in texto:
        return f"🎥 {valor} (vídeo)"
    elif "página" in texto or texto.isdigit():
        return f"📚 {valor} (artigo)"
    return f"⏱️ {valor}"

if "Descricao_duracao" not in df_odas.columns:
    df_odas["Descricao_duracao"] = df_odas["Duração/Tamanho"].apply(classificar_duracao)

def encontrar_rubrica(pontuacao, dimensao, subdimensao):
    candidatos = df_rubricas[
        (df_rubricas['dimensao'] == dimensao) &
        (df_rubricas['subdimensao'] == subdimensao) &
        (df_rubricas['faixa_total_min'] <= pontuacao) &
        (df_rubricas['faixa_total_max'] >= pontuacao)
    ]
    if candidatos.empty:
        return None, None, None

    rubrica_numero = candidatos.iloc[0]['rubrica_numero']
    rubrica_nome = candidatos.iloc[0]['rubrica_nome']

    faixa_escolhida = candidatos[
        (candidatos['subfaixa_min'] <= pontuacao) &
        (candidatos['subfaixa_max'] >= pontuacao)
    ]
    if faixa_escolhida.empty:
        return rubrica_numero, rubrica_nome, None

    tipo_faixa = faixa_escolhida.iloc[0]['tipo_faixa']
    return rubrica_numero, rubrica_nome, tipo_faixa

# === TEXTO PARA EMBEDDING (sem formatação) ===
def gerar_texto_devolutiva_rico(pontuacao, dimensao, subdimensao):
    rubrica_numero, rubrica_nome, tipo_faixa = encontrar_rubrica(pontuacao, dimensao, subdimensao)

    if not rubrica_numero or not tipo_faixa:
        return None

    rubrica_nome_completo = f"{rubrica_nome} – Nível {tipo_faixa}"

    devolutiva = df_devolutivas[
        (df_devolutivas['Dimensão'] == dimensao) &
        (df_devolutivas['Subdimensão'] == subdimensao) &
        (df_devolutivas['Rubrica numero'] == rubrica_numero) &
        (df_devolutivas['Rubrica nome'] == rubrica_nome_completo)
    ]

    if devolutiva.empty:
        return None

    item = devolutiva.iloc[0]

    texto = f"""
            Dimensão: {dimensao}
            Subdimensão: {subdimensao}
            Rubrica: Rubrica {rubrica_numero} - {rubrica_nome}
            Nível: {tipo_faixa}

            Seus pontos fortes:
            {item['Pontos fortes']}

            O que fazer para avançar:
            {item['O que fazer para avançar']}

            Necessidades formativas:
            {item['Necessidades formativas']}
            """.strip()

    return texto

# === TEXTO PARA EXIBIR (bonito com Markdown e emojis) ===
def gerar_texto_devolutiva_markdown(pontuacao, dimensao, subdimensao):
    rubrica_numero, rubrica_nome, tipo_faixa = encontrar_rubrica(pontuacao, dimensao, subdimensao)

    if not rubrica_numero or not tipo_faixa:
        return None

    rubrica_nome_completo = f"{rubrica_nome} – Nível {tipo_faixa}"

    devolutiva = df_devolutivas[
        (df_devolutivas['Dimensão'] == dimensao) &
        (df_devolutivas['Subdimensão'] == subdimensao) &
        (df_devolutivas['Rubrica numero'] == rubrica_numero) &
        (df_devolutivas['Rubrica nome'] == rubrica_nome_completo)
    ]

    if devolutiva.empty:
        return None

    item = devolutiva.iloc[0]

    return f"""
            ## 📄 **Devolutiva personalizada:**

            🔢 **Pontuação:** {pontuacao}
            📂 **Dimensão:** {dimensao}
            📁 **Subdimensão:** {subdimensao}
            🏷️ **Rubrica:** Rubrica {rubrica_numero} - {rubrica_nome}
            📊 **Nível:** {tipo_faixa}

            ---

            ✅ **Seus pontos fortes:**
            {item['Pontos fortes']}

            ---

            📈 **O que fazer para avançar:**
            {item['O que fazer para avançar']}

            ---

            📚 **Necessidades formativas:**
            {item['Necessidades formativas']}
            """.strip()

def gerar_embedding_para_rag(texto: str) -> np.ndarray:
    embedding = modelo.encode([texto])
    embedding = embedding / np.linalg.norm(embedding, axis=1, keepdims=True)
    return embedding

def interpretar_similaridade(valor):
    if valor >= 0.80:
        return "🔥 Altamente relevante"
    elif valor >= 0.65:
        return "✅ Relevante"
    elif valor >= 0.50:
        return "🧐 Moderadamente relevante"
    else:
        return "🔍 Pouco relevante"

def gerar_card_material(row, i):
    titulo = row.get("Título", "Sem título")
    resumo = re.sub(r"<[^>]+>", "", str(row.get("Resumo", "Sem resumo disponível")).strip())
    suporte = row.get("Suporte", "Não informado")
    tipo = row.get("Tipo", "Não informado")
    dimensao = row.get("Dimensões", "Não informado")
    duracao = row.get("Descricao_duracao", "⏱️ Duração não informada")
    link_real = str(row.get("Fonte", "#")).strip()
    if link_real.lower() == "nan" or link_real == "":
        link_real = "#"
    sim = float(row['distância'])
    interpretacao = interpretar_similaridade(sim)

    return f"""
**{i+1}. [{titulo}]({link_real})**
- 📝 **Resumo:** {resumo}
- 📎 **Tipo:** {suporte} | **Subtipo:** {tipo}
- 🔍 **Dimensão:** {dimensao}
- ⏱️ **Duração:** {duracao}
- 📏 **Similaridade:** {sim:.4f} – *{interpretacao}*

---
"""

# === 4. INTERFACE INTERATIVA ===
dimensoes_unicas = df_devolutivas['Dimensão'].unique().tolist()

dropdown_dimensao = widgets.Dropdown(
    options=dimensoes_unicas,
    description='Dimensão:',
    value=dimensoes_unicas[0],
    style={'description_width': 'initial'}
)

dropdown_subdimensao = widgets.Dropdown(
    options=[],
    description='Subdimensão:',
    style={'description_width': 'initial'}
)

def atualizar_subdimensoes(*args):
    subdimensoes = df_devolutivas[
        df_devolutivas['Dimensão'] == dropdown_dimensao.value
    ]['Subdimensão'].unique().tolist()
    dropdown_subdimensao.options = subdimensoes
    if subdimensoes:
        dropdown_subdimensao.value = subdimensoes[0]

dropdown_dimensao.observe(atualizar_subdimensoes, names='value')
atualizar_subdimensoes()

entrada_pontuacao = widgets.IntSlider(
    value=17,
    min=0,
    max=51,
    step=1,
    description='Pontuação:',
    style={'description_width': 'initial'},
    continuous_update=False
)

botao_gerar = widgets.Button(description='Gerar devolutiva', button_style='success')
output = widgets.Output()

def ao_clicar_botao(b):
    with output:
        output.clear_output()

        texto_rico = gerar_texto_devolutiva_rico(
            pontuacao=entrada_pontuacao.value,
            dimensao=dropdown_dimensao.value,
            subdimensao=dropdown_subdimensao.value
        )
        texto_markdown = gerar_texto_devolutiva_markdown(
            pontuacao=entrada_pontuacao.value,
            dimensao=dropdown_dimensao.value,
            subdimensao=dropdown_subdimensao.value
        )

        if texto_rico is None or texto_markdown is None:
            display(Markdown("⚠️ Não foi possível gerar devolutiva para essa pontuação."))
            return

        display(Markdown(texto_markdown))

        embedding = gerar_embedding_para_rag(texto_rico)
        distancias, indices = index.search(np.array(embedding).astype("float32"), 50)
        resultados = df_odas.iloc[indices[0]].copy()
        resultados["distância"] = distancias[0]

        resultados = resultados[resultados["Idiomas"].str.contains("português", case=False, na=False)]

        artigos = resultados[resultados["Suporte"].str.contains("Artigo|Texto|Livro", case=False, na=False)]
        videos = resultados[resultados["Suporte"].str.contains("Vídeo|Aula|Curso", case=False, na=False)]

        markdown = "## 📚 **Materiais recomendados – Artigos:**\n\n"
        for i, row in artigos.iterrows():
            markdown += gerar_card_material(row, i)

        markdown += "\n\n## 🎥 **Materiais recomendados – Vídeos:**\n\n"
        for i, row in videos.iterrows():
            markdown += gerar_card_material(row, i)

        display(Markdown(markdown))

botao_gerar.on_click(ao_clicar_botao)

display(dropdown_dimensao, dropdown_subdimensao, entrada_pontuacao, botao_gerar, output)


Dropdown(description='Dimensão:', options=('Dimensão pedagógica', 'Dimensão pessoal-relacional'), style=Descri…

Dropdown(description='Subdimensão:', options=('Planejamento pedagógico', 'Implementação do processo de ensino …

IntSlider(value=17, continuous_update=False, description='Pontuação:', max=51, style=SliderStyle(description_w…

Button(button_style='success', description='Gerar devolutiva', style=ButtonStyle())

Output()

In [ ]:
df_odas2 = pd.read_excel(caminho_xlsx)

In [ ]:
# Verifica se a coluna 'Tipo' existe no DataFrame
if 'Tipo' in df_odas2.columns:
    # Conta os tipos únicos de nomes na coluna 'Tipo'
    contagem_tipos = df_odas2['Tipo'].value_counts()

    # Imprime a contagem
    print("Contagem de tipos de nomes na coluna 'Tipo':")
    print(contagem_tipos)

    # Imprime os tipos únicos
    print("\nTipos de nomes únicos na coluna 'Tipo':")
    print(df_odas2['Tipo'].unique())
else:
    print("A coluna 'Tipo' não existe no DataFrame.")


Contagem de tipos de nomes na coluna 'Tipo':
Tipo
Artigo                                               446
Videoconferência                                     270
Periódico                                            221
Vídeo Educativo                                      204
Manuais, materiais e textos de orientação e apoio    199
Dissertação                                          175
Entrevista                                           150
Relatório                                            148
Material Jornalístico                                147
Curso                                                105
Evento                                                91
Tese                                                  49
Livro                                                 40
Boletim                                               38
Documentário                                          34
Legislação e Instrumentos Legais                      32
E-book                                

In [ ]:
with open("metadados_odas_stellav5.pkl", "wb") as f:
    pickle.dump(metadados, f)

faiss.write_index(index, "odas_index_stellav5.faiss")